In [1]:
import pandas as pd
from collections import defaultdict
import re
import numpy as np
import sys
sys.path.insert(0, '../../base_codes')
sys.path.insert(0, '../../data_specific_codes')
sys.path.insert(0, '../../utility_codes')


from RE_init import *
from main_functions import *
from utility_functions import *

def read_df_rel():
    #based_dir = '../../../../data/GoodReads/'
    based_dir = '../../../../data/Tweets/all_tweets_with_extractions_from_spade/'
    file_input_name = 'merged_relations_-1.csv'
    file_input = based_dir + file_input_name    
    ff = open(file_input)
    delim=","
    df = pd.read_csv(file_input,delimiter=delim,header=0)        
    return df

df_rels = read_df_rel()

In [3]:
df_rels

,tweet_id,tweet_posted_time,tweet_author_id,tweet_author_location,tweet_country,city,tweet_lang,lat,lon,ALTLAT,...,arg2,type,pattern,arg1_with_pos,rel_with_pos,arg2_with_pos,arg1_prepositions,rel_prepositions,arg2_prepositions,annotation
0,2005:616033582348374016,2015-07-01T00:00:26.000Z,20562637,"New York, NY",NaN,NaN,en,NaN,NaN,NaN,...,an early {lead},SVO,"(nsubj, verb, dobj)",SamsungPay-NNP-2,Why-WRB-1 could-MD-3 gain-VB-4,an-DT-5 early-JJ-6 lead-NN-7,NaN,NaN,REASON: prep_in TEXT: mobile {payments} --,"{'dep_parse': 'advmod(gain-4, Why-1)\nnsubj(ga..."
1,2005:616033586781814785,2015-07-01T00:00:27.000Z,8841372,"New York, NY",NaN,NaN,en,NaN,NaN,NaN,...,an early {lead},SVO,"(nsubj, verb, dobj)",SamsungPay-NNP-2,Why-WRB-1 could-MD-3 gain-VB-4,an-DT-5 early-JJ-6 lead-NN-7,NaN,NaN,REASON: prep_in TEXT: mobile {payments} --,"{'dep_parse': 'advmod(gain-4, Why-1)\nnsubj(ga..."
2,2005:616033610773041152,2015-07-01T00:00:32.000Z,1891949672,NaN,NaN,NaN,en,NaN,NaN,NaN,...,an early {lead},SVO,"(nsubj, verb, dobj)",SamsungPay-NNP-2,Why-WRB-1 could-MD-3 gain-VB-4,an-DT-5 early-JJ-6 lead-NN-7,NaN,NaN,REASON: prep_in TEXT: mobile {payments} --,"{'dep_parse': 'advmod(gain-4, Why-1)\nnsubj(ga..."
3,2005:616033653701890048,2015-07-01T00:00:43.000Z,36204878,"Marietta, GA",NaN,NaN,en,NaN,NaN,NaN,...,an early {lead},SVO,"(nsubj, verb, dobj)",SamsungPay-NNP-2,Why-WRB-1 could-MD-3 gain-VB-4,an-DT-5 early-JJ-6 lead-NN-7,NaN,NaN,REASON: prep_in TEXT: mobile {payments} --,"{'dep_parse': 'advmod(gain-4, Why-1)\nnsubj(ga..."
4,2005:616033660068761600,2015-07-01T00:00:44.000Z,63201802,"Mxico, D.F.",NaN,NaN,en,NaN,NaN,NaN,...,an early {lead},SVO,"(nsubj, verb, dobj)",SamsungPay-NNP-2,Why-WRB-1 could-MD-3 gain-VB-4,an-DT-5 early-JJ-6 lead-NN-7,NaN,NaN,REASON: prep_in TEXT: mobile {payments} --,"{'dep_parse': 'advmod(gain-4, Why-1)\nnsubj(ga..."
5,2005:616033679169687552,2015-07-01T00:00:49.000Z,2805624278,"Florida, USA",NaN,NaN,en,NaN,NaN,NaN,...,an early {lead},SVO,"(nsubj, verb, dobj)",SamsungPay-NNP-2,Why-WRB-1 could-MD-3 gain-VB-4,an-DT-5 early-JJ-6 lead-NN-7,NaN,NaN,REASON: prep_in TEXT: mobile {payments} --,"{'dep_parse': 'advmod(gain-4, Why-1)\nnsubj(ga..."
6,2005:616033691786084352,2015-07-01T00:00:52.000Z,2720373876,"Los Angeles, CA",NaN,NaN,en,NaN,NaN,NaN,...,an early {lead},SVO,"(nsubj, verb, dobj)",SamsungPay-NNP-2,Why-WRB-1 could-MD-3 gain-VB-4,an-DT-5 early-JJ-6 lead-NN-7,NaN,NaN,REASON: prep_in TEXT: mobile {payments} --,"{'dep_parse': 'advmod(gain-4, Why-1)\nnsubj(ga..."
7,2005:616033710895407104,2015-07-01T00:00:56.000Z,3299992401,"Manchester, England",NaN,NaN,en,NaN,NaN,NaN,...,an early {lead},SVO,"(nsubj, verb, dobj)",SamsungPay-NNP-2,Why-WRB-1 could-MD-3 gain-VB-4,an-DT-5 early-JJ-6 lead-NN-7,NaN,NaN,REASON: prep_in TEXT: mobile {payments} --,"{'dep_parse': 'advmod(gain-4, Why-1)\nnsubj(ga..."
8,2005:616034001741082624,2015-07-01T00:02:06.000Z,199620260,Santarcangelo di Romagna,NaN,NaN,en,NaN,NaN,NaN,...,an early {lead},SVO,"(nsubj, verb, dobj)",SamsungPay-NNP-2,Why-WRB-1 could-MD-3 gain-VB-4,an-DT-5 early-JJ-6 lead-NN-7,NaN,NaN,REASON: prep_in TEXT: mobile {payments} --,"{'dep_parse': 'advmod(gain-4, Why-1)\nnsubj(ga..."
9,2005:616034053255360512,2015-07-01T00:02:18.000Z,2794526676,Texas,NaN,NaN,en,NaN,NaN,NaN,...,a {drink},SVO,"(nsubj, verb, dobj)",someone-NN-5,should-MD-6 buy-VB-7,a-DT-9 drink-NN-10,NaN,NaN,NaN,"{'dep_parse': 'amod(Time-3, Happy-1)\nnn(Time-..."


In [2]:
'''
if NODE_SELECTION:
    # get the list of different versions of an entity. Example : parents,parent,i,we -> parents
    entity_versions = get_entity_versions(DATA_SET)    
    df_simp = get_simp_df(df_rels.copy(),entity_versions)  
    selected_nodes = entity_versions.keys()
    df_rels_selected = filter_nodes_OR(df_simp.copy(),source='arg1',target='arg2',selected_nodes = selected_nodes)
    g_arg = create_argument_multiGraph(df_rels_selected.copy(),source='arg1',target='arg2',edge_attr = 'rel')
    if SAVE_GEFX:
        nx.write_gexf(g_arg, output_dir + file_input_name + "_" + "g_arg_selected_"+str(MAX_ITERATION)+"_"+str(time.time())+".gexf")
    #plot_argument_graph(g_arg)
    if SAVE_PAIRWISE_RELS:
        file_loc = output_dir + file_input_name + "_" + "pairwise_rels_selected_"+str(MAX_ITERATION)+"_"+DATA_SET+".txt"
        save_pairwise_rels(file_loc,g_arg,print_option=True)      

g_arg = create_argument_multiGraph(df_rels.copy(),source='arg1',target='arg2',edge_attr = 'rel')
if SAVE_GEFX:
    nx.write_gexf(g_arg, output_dir + file_input_name + "_" + "g_arg_"+str(MAX_ITERATION)+"_"+str(time.time())+".gexf")
#plot_argument_graph(g_arg)
if SAVE_PAIRWISE_RELS:
    file_loc = output_dir + file_input_name + "_"  + "pairwise_rels_"+str(MAX_ITERATION)+"_"+DATA_SET+".txt"
    save_pairwise_rels(file_loc,g_arg,print_option=True)    
    
#if __name__ == "__main__":
#    main(sys.argv[1:])
'''

'\nif NODE_SELECTION:\n    # get the list of different versions of an entity. Example : parents,parent,i,we -> parents\n    entity_versions = get_entity_versions(DATA_SET)    \n    df_simp = get_simp_df(df_rels.copy(),entity_versions)  \n    selected_nodes = entity_versions.keys()\n    df_rels_selected = filter_nodes_OR(df_simp.copy(),source=\'arg1\',target=\'arg2\',selected_nodes = selected_nodes)\n    g_arg = create_argument_multiGraph(df_rels_selected.copy(),source=\'arg1\',target=\'arg2\',edge_attr = \'rel\')\n    if SAVE_GEFX:\n        nx.write_gexf(g_arg, output_dir + file_input_name + "_" + "g_arg_selected_"+str(MAX_ITERATION)+"_"+str(time.time())+".gexf")\n    #plot_argument_graph(g_arg)\n    if SAVE_PAIRWISE_RELS:\n        file_loc = output_dir + file_input_name + "_" + "pairwise_rels_selected_"+str(MAX_ITERATION)+"_"+DATA_SET+".txt"\n        save_pairwise_rels(file_loc,g_arg,print_option=True)      \n\ng_arg = create_argument_multiGraph(df_rels.copy(),source=\'arg1\',target=\

In [6]:
entity_versions = get_entity_versions('twitter')    
df_simp = get_simp_df(df_rels.copy(),entity_versions)  
selected_nodes = entity_versions.keys()
df_rels_selected = filter_nodes_OR(df_simp.copy(),source='arg1',target='arg2',selected_nodes = selected_nodes)

In [19]:
output_dir = '../../../../data/Tweets/all_tweets_with_extractions_from_spade/'
pkl_loc = output_dir + "df_simp.pkl"
CREATE_SAVE_PICKLE=False#True
if CREATE_SAVE_PICKLE:
    df_simp = get_simp_df(df_rels.copy(),entity_versions)  
    df_simp.to_pickle(pkl_loc)
else:
    df_simp = pd.read_pickle(pkl_loc)
df_simp

IOError: [Errno 2] No such file or directory: '../../../../data/Tweets/all_tweets_with_extractions_from_spade/df_rels_selected.pkl'

In [18]:
df_simp.to_pickle(pkl_loc)

NameError: name 'pkl_loc' is not defined

In [8]:
df_rels_selected

,tweet_id,tweet_posted_time,tweet_author_id,tweet_author_location,tweet_country,city,tweet_lang,lat,lon,ALTLAT,...,arg2,type,pattern,arg1_with_pos,rel_with_pos,arg2_with_pos,arg1_prepositions,rel_prepositions,arg2_prepositions,annotation
51,2005:616052258191577088,2015-07-01T01:14:38.000Z,566697301,NJ,NaN,NaN,en,NaN,NaN,NaN,...,googlewallet,SVO,"(nsubj, verb, dobj)",you-PRP-3,use-VBP-4,googlewallet-.-5,NaN,NaN,NaN,"{'dep_parse': 'nsubj(believe-2, i-1)\nroot(ROO..."
54,2005:616053600893145088,2015-07-01T01:19:58.000Z,2247334022,Wagging in MI,NaN,NaN,en,NaN,NaN,NaN,...,PayPal,SVO,"(nsubj, verb, dobj)",I-PRP-2,get-VBP-3,a-DT-4 PayPal-NNP-5,NaN,NaN,NaN,"{'dep_parse': 'advmod(will-9, once-1)\nnsubj(g..."
57,2005:616057483015155712,2015-07-01T01:35:24.000Z,32439880,"Chicago, IL",NaN,NaN,en,NaN,NaN,NaN,...,own mobile payment {device},SVO,"(nsubj, verb, dobj)",Barclays-NNP-1 Bank-NNP-2,is-VBZ-6 developing-VBG-7,own-JJ-9 mobile-JJ-10 payment-NN-11 device-NN-12,REASON: prep_in TEXT: the {UK} --,NaN,NaN,"{'dep_parse': 'nn(Bank-2, Barclays-1)\nnsubj(d..."
61,2005:616064196317306881,2015-07-01T02:02:04.000Z,488597347,NaN,NaN,NaN,en,NaN,NaN,NaN,...,PayPal,SVO,"(nsubj, verb, dobj)",I-PRP-1,got-VBD-2,PayPal-NNP-3,NaN,NaN,NaN,"{'dep_parse': 'nsubj(got-2, I-1)\nroot(ROOT-0,..."
69,2005:616072613857763328,2015-07-01T02:35:31.000Z,62897798,"Albuquerque, NM",NaN,NaN,en,NaN,NaN,NaN,...,{me},SVO,"(nsubj, verb, dobj)",googlewallet-NN-1,saved-VBD-3 now-RB-6,me-PRP-4,NaN,NaN,NaN,"{'dep_parse': 'nsubj(saved-3, googlewallet-1)\..."
118,2005:616101128997093376,2015-07-01T04:28:50.000Z,270018506,"London, England",NaN,NaN,en,NaN,NaN,NaN,...,banks,SVO,"(nsubj, verb, dobj)",I-PRP-10,'m-VBP-11 finding-VBG-12,another-DT-13 bank-NN-14,NaN,NaN,NaN,"{'dep_parse': ""mark(supporting-6, as-1)\nnsubj..."
163,2005:616108389328928769,2015-07-01T04:57:41.000Z,860947765,NaN,NaN,NaN,en,NaN,NaN,NaN,...,own custom {video},SVO,"(nsubj, verb, dobj)",Kaylawalton36yahoo-NNP-1 .com-NN-2 4-CD-3 PayP...,get-VB-9,own-JJ-11 custom-NN-12 video-NN-13,NaN,NaN,NaN,"{'dep_parse': 'nn(PayPal-4, Kaylawalton36yahoo..."
164,2005:616108389328928769,2015-07-01T04:57:41.000Z,860947765,NaN,NaN,NaN,en,NaN,NaN,NaN,...,{ht},SVO,"(nsubj, verb, dobj)",Kaylawalton36yahoo-NNP-1 .com-NN-2 4-CD-3 PayP...,get-VB-9,ht-NN-17,NaN,NaN,NaN,"{'dep_parse': 'nn(PayPal-4, Kaylawalton36yahoo..."
211,2005:616130608700489728,2015-07-01T06:25:58.000Z,22785567,"America, China, Singapore & UK",NaN,NaN,en,NaN,NaN,NaN,...,bpay mobile {payments} compete,SVO,"(nsubj, verb, dobj)",Barclays-NNP-1 UK-NNP-2,is-VBZ-3 relaunching-VBG-4,bPay-JJ-5 mobile-JJ-6 payments-NNS-7 compete-VB-9,NaN,NaN,NaN,"{'dep_parse': 'nn(UK-2, Barclays-1)\nnsubj(rel..."
224,2005:616139462276419584,2015-07-01T07:01:09.000Z,381985266,NaN,NaN,NaN,en,NaN,NaN,NaN,...,the US,SVO,"(nsubj, verb, dobj)",I-PRP-1,thought-VBD-2,us-PRP-3,NaN,NaN,NaN,"{'dep_parse': 'nsubj(thought-2, I-1)\nroot(ROO..."


In [9]:
df = df_rels_selected[["arg1", "rel", "arg2", "arg1_prepositions","rel_prepositions","arg2_prepositions"]]

In [10]:
df

,arg1,rel,arg2,arg1_prepositions,rel_prepositions,arg2_prepositions
51,{you},{use},googlewallet,NaN,NaN,NaN
54,{i},{get},PayPal,NaN,NaN,NaN
57,banks,is {developing},own mobile payment {device},REASON: prep_in TEXT: the {UK} --,NaN,NaN
61,{i},{got},PayPal,NaN,NaN,NaN
69,googlewallet,{saved} now,{me},NaN,NaN,NaN
118,{i},'m {finding},banks,NaN,NaN,NaN
163,PayPal,{get},own custom {video},NaN,NaN,NaN
164,PayPal,{get},{ht},NaN,NaN,NaN
211,the UK,is {relaunching},bpay mobile {payments} compete,NaN,NaN,NaN
224,{i},{thought},the US,NaN,NaN,NaN


In [13]:
df[np.logical_and(df['arg1'].str.contains("ApplePay"),df['arg2'].str.contains("Barclays"))]

,arg1,rel,arg2,arg1_prepositions,rel_prepositions,arg2_prepositions


In [16]:
df[df['arg1'].str.contains("applepay")]

,arg1,rel,arg2,arg1_prepositions,rel_prepositions,arg2_prepositions
1702,{applepay},{enters},the UK,NaN,NaN,NaN
1706,{applepay},{enters},the UK,NaN,NaN,NaN
1710,{applepay},{enters},the UK,NaN,NaN,NaN
1714,{applepay},{enters},the UK,NaN,NaN,NaN
1718,{applepay},{enters},the UK,NaN,NaN,NaN
1722,{applepay},{enters},the UK,NaN,NaN,NaN
1727,{applepay},{enters},the UK,NaN,NaN,NaN
1731,{applepay},{enters},the UK,NaN,NaN,NaN
1736,{applepay},{enters},the UK,NaN,NaN,NaN
1740,{applepay},{enters},the UK,NaN,NaN,NaN
